<img width="10%" alt="HubSpot" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/HubSpot.png" style="border-radius: 15%">

# HubSpot - Update linkedinbio from google
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/HubSpot/HubSpot_Update_linkedinbio_from_google.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=HubSpot+-+Update+linkedinbio+from+google:+Error+short+description">Bug report</a>

**Tags:** #hubspot #crm #sales #contact #naas_drivers #googlesearch #scheduler #naas #automation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-04-12 (Created: 2022-02-21)

**Description:** This notebook update HubSpot linkedin URL based on Google Search with firstname and lastname.

## Input

### Import libraries

In [ ]:
from naas_drivers import hubspot
import naas
import pandas as pd
from googlesearch import search
import time
import re

### Setup HubSpot
ðŸ‘‰ Starting November 30, 2022, HubSpot API keys no longer enable access to HubSpot APIs, so in Naas version 2.8.3 and above, you need [create a private app and use the access token](https://developers.hubspot.com/docs/api/private-apps).

In [ ]:
# Enter Your Access Token
HS_ACCESS_TOKEN = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"

## Model

### Get all contacts in Hubspot

In [ ]:
properties_list = [
    "hs_object_id",
    "firstname",
    "lastname",
    "linkedinbio",
    "naas_user",
    "naas_user_plan"
]
hubspot_contacts = hubspot.connect(HS_ACCESS_TOKEN).contacts.get_all(properties_list)
hubspot_contacts

### Filter to get linkedinbio "Not Defined" and "firstname" and "lastname" defined

In [ ]:
df_to_update = hubspot_contacts.copy()

# Cleaning
df_to_update = df_to_update.fillna("Not Defined")

# Filter on "Not defined"
df_to_update = df_to_update[
    (df_to_update.firstname != "Not Defined") &
    (df_to_update.lastname != "Not Defined") &
    (df_to_update.linkedinbio == "Not Defined") &
    (df_to_update.naas_user == "Yes") &
    (df_to_update.naas_user_plan == "NAAS_FREE")
].sort_values(by="lastmodifieddate", ascending=False).reset_index(drop=True)

df_to_update

### Search bio in Google with firstname and lastname

In [ ]:
def get_bio(firstname, lastname):
    # Init linkedinbio
    linkedinbio = None

    # Create query
    query = f"{firstname}+{lastname}+Linkedin"
    print("Google query: ", query)

    # Search in Google
    for i in search(query, tld="com", num=10, stop=10, pause=2):
        pattern = "https:\/\/.+.linkedin.com\/in\/.([^?])+"
        result = re.search(pattern, i)

        # Return value if result is not None
        if result != None:
            linkedinbio = result.group(0).replace(" ", "")
            return linkedinbio
        else:
            time.sleep(2)
    return linkedinbio

## Output

In [ ]:
for _, row in df_to_update.iterrows():
    firstname = row.firstname
    lastname = row.lastname

    # Get linkedinbio
    linkedinbio = get_bio(firstname, lastname)
    df_to_update.loc[_, "linkedinbio"] = linkedinbio
    
    # Init data
    data = {}

    # Get data
    hs_object_id = row.hs_object_id
    linkedinbio = row.linkedinbio

    # Update LK Bio
    if linkedinbio != None:
        data = {"properties": {"linkedinbio": linkedinbio}}
    try:
        hubspot.connect(HS_ACCESS_TOKEN).contacts.patch(hs_object_id, data)
    except Exception as e:
        print(e)

df_to_update